In [1]:
import bpy
import bmesh
import numpy as np
import matplotlib.pyplot as plt
import math



import trimesh
import pyrender
import bmesh
import mathutils
import os
from util import get_contour,bounds,rotate_2d_np,compute_bbox
from util import contour2poly

from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.geometry import Point, Polygon,LineString
import time

import random
import networkx as nx

import networkx as nx
import numpy as np
from shapely.geometry import Point, LineString
from shapely.prepared import prep
from PIL import Image
from scipy.spatial.transform import Rotation as R
import csv


In [2]:
def triangulate_object(obj):
    # Get the mesh data
    mesh = obj.data

    # Create a BMesh from the mesh
    bm = bmesh.new()
    bm.from_mesh(mesh)

    # Triangulate all faces
    bmesh.ops.triangulate(bm, faces=bm.faces[:])

    # Write the bmesh back to the mesh and free bmesh
    bm.to_mesh(mesh)
    bm.free()

def blender_mesh_to_trimesh(obj):
    triangulate_object(obj)
    mesh = obj.data

    vertices_local = [v.co for v in mesh.vertices]
    vertices_world = np.array([
        (obj.matrix_world @ mathutils.Vector(v)).to_tuple()
        for v in vertices_local
    ])

    faces = np.array([f.vertices[:] for f in mesh.polygons if len(f.vertices) == 3])
    tm = trimesh.Trimesh(vertices=vertices_world, faces=faces, process=False)
    return tm

def mesh_to_xy_polygon(mesh: trimesh.Trimesh) -> Polygon:
    # Project vertices onto XY plane by dropping Z coordinate

    projected_verts = mesh.vertices[:, :2]  # (N, 2)
    
    # Get the 2D projected faces as polygons
    faces = mesh.faces
    face_polys = []
    for f in faces:
        pts = projected_verts[f]  # 3 points for one triangle
        if not trimesh.triangles.area(pts.reshape(1, 3, 2))[0] > 1e-8:
            continue  # Skip degenerate triangles
        face_polys.append(Polygon(pts))
    
    # Union all triangle polygons to get a single polygon
    return unary_union(face_polys)

def get_unique_color(index):
    """Convert an integer index into a unique RGB color"""
    r = (index & 255)
    g = (index >> 8) & 255
    b = (index >> 16) & 255
    return [r, g, b, 255]  # RGBA

def is_valid_point(x, y,prep_room,prep_obstacles):
    p = Point(x, y)
    # Must be inside room AND not inside any obstacle
    return prep_room.contains(p) and not prep_obstacles.intersects(p)

def get_nearest_grid_node(x, y, graph):
    # Find the node in the graph closest to the requested real-world coordinate
    nodes = np.array(graph.nodes)
    dist = np.sum((nodes - np.array([x, y]))**2, axis=1)
    nearest_idx = np.argmin(dist)
    return tuple(nodes[nearest_idx])

def compute_path_segment(start_x, start_y, end_x, end_y,G):
    start_node = get_nearest_grid_node(start_x, start_y, G)
    end_node = get_nearest_grid_node(end_x, end_y, G)
    
    try:
        # Dijkstra (weighted shortest path)
        path_nodes = nx.shortest_path(G, source=start_node, target=end_node, weight='weight')
        return path_nodes
    except nx.NetworkXNoPath:
        print(f"No path found between ({start_x},{start_y}) and ({end_x},{end_y})")
        return []

def get_camera_rotation_matrix(position, yaw, pitch):
    """
    Returns a world matrix for a Blender camera (which points -Z)
    oriented according to standard Z-up Yaw/Pitch.
    """
    # 1. Calculate the Look Direction Vector from Yaw/Pitch
    # Assuming Yaw starts at X-axis (0) and rotates around Z
    # Pitch is elevation from horizon
    import math
    
    # Direction vector (Forward)
    # Note: Adjust these sin/cos if your pitch/yaw definition differs!
    # Here: Yaw=0 -> +X, Pitch=0 -> Horizon
    dir_x = math.cos(yaw) * math.cos(pitch)
    dir_y = math.sin(yaw) * math.cos(pitch)
    dir_z = math.sin(pitch)

    direction = mathutils.Vector((dir_x, dir_y, dir_z))
    
    # 2. Blender Camera Orientation Correction
    # Blender Camera: -Z is Forward, +Y is Up.
    # We want -Z to align with 'direction', and +Y to align with World Up (Z)
    
    rot_quat = direction.to_track_quat('-Z', 'Y')
    
    # Create the full 4x4 matrix
    mat_loc = mathutils.Matrix.Translation(position)
    mat_rot = rot_quat.to_matrix().to_4x4()
    
    return mat_loc @ mat_rot

In [3]:
def camera_traj(file_name="scene (1).blend",
                room_type="living-room_0/0.ceiling",
                OUTPUT_DIR = "//render_output/",
                FRAME_FILENAME = "frame_",
                VIDEO_FILENAME = "trajectory_video.mp4",
                GRID_RES = 0.2,
                ROTATION_STEPS = 30,
                height=1.5
               ):

    
    exclude_obj= ["PointLamp", "Window", "CeilingLight","Door","Carnivore"]
    bpy.ops.wm.open_mainfile(filepath=file_name)
    start_all = time.time()
    
    for ob in bpy.data.objects:
        if room_type in ob.name:
            x_vals,y_vals = get_contour(ob)
    poly_room=contour2poly(x_vals,y_vals)
    from shapely.affinity import translate
    for ob in bpy.data.objects:
        if room_type in ob.name:
            offset_x=ob.location.x
            offset_y=ob.location.y
    
    poly_room = translate(poly_room, xoff=offset_x, yoff=offset_y)



    min_dist = float('inf')
    selected_door_name = None
    
    for ob in bpy.data.objects:
        # Check for "door" (case-insensitive)
        if "doorfactory" in ob.name.lower():
            # Get global location
            
            global_loc = ob.matrix_world.translation
            door_point = Point(global_loc.x, global_loc.y)
           # print(door_point)
            
            # Calculate distance to the room boundary
            dist = poly_room.boundary.distance(door_point)
            
            # Update if this is the closest door found so far
            if dist < min_dist:
                min_dist = dist
                selected_door_name = ob.name
    
    # Output the result
    #print(selected_door_name)
    for ob in bpy.data.objects:
        # Check for "door" (case-insensitive)
        if selected_door_name in ob.name:
            global_loc = ob.matrix_world.translation
            x_start=global_loc.x
            y_start=global_loc.y

    camera_data = bpy.context.scene.camera.data
    if camera_data.type == 'PERSP':
    
        sensor_width = camera_data.sensor_width
        sensor_height = camera_data.sensor_height
        focal_length = camera_data.lens
        hor_fov_rad = 2 * math.atan((sensor_width / 2) / focal_length)
        ver_fov_rad = 2 * math.atan((sensor_height / 2) / focal_length)
        #print(ver_fov_rad,hor_fov_rad)

    #simplify 3d mesh in blender
    target_poly_density=100000
    org_poly_num=[]
    simp_poly_num=[]
    for obj in bpy.data.objects:
        if obj.type == 'MESH' and "spawn_asset" in obj.name and all(x not in obj.name for x in exclude_obj):
            cur_density=len(obj.data.polygons)/sum(p.area for p in obj.data.polygons)
            org_poly_num.append(len(obj.data.polygons))
            if cur_density>target_poly_density:
                obj.select_set(True)
                bpy.context.view_layer.objects.active = obj
                
                mod = obj.modifiers.new(name='Decimate', type='DECIMATE')
                mod.ratio = target_poly_density/cur_density
                mod.use_collapse_triangulate = True  # Optional, helps preserve shape
                bpy.ops.object.mode_set(mode='OBJECT')
                bpy.ops.object.modifier_apply(modifier=mod.name)
            simp_poly_num.append(len(obj.data.polygons))
            
            #print(cur_density,obj.name)

    trimesh_obj_list=[]
    cnt=0
    num_faces=[]
    obj_name_list=[]
    obj_dimensions_data = []
    for obj in bpy.data.objects:
        if obj.type == 'MESH' and "spawn_asset" in obj.name and all(x not in obj.name for x in exclude_obj):
            
            trimesh_obj=blender_mesh_to_trimesh(obj)
            trimesh_obj_list.append(trimesh_obj)
    
            #num_verts.append(trimesh_obj.vertices.shape[0])
            #org_num_faces.append(len(trimesh_obj.faces))
            num_faces.append(len(trimesh_obj.faces))
            
            #print(f"{cnt} Faces org: {len(trimesh_obj.faces)},{obj.name}")
            obj_name_list.append(obj.name)
            cnt=cnt+1

            dims = trimesh_obj.extents
        
            obj_dimensions_data.append({
                "name": obj.name,
                "width_x": dims[0],
                "length_y": dims[1],
                "height_z": dims[2]
            })

    csv_path = OUTPUT_DIR+"/"+"object_bbox_dimensions.csv"

    try:
        with open(csv_path, 'w', newline='') as f:
            # Define headers
            fieldnames = ["name", "width_x", "length_y", "height_z"]
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            
            writer.writeheader()
            writer.writerows(obj_dimensions_data)
            
        print(f"Successfully saved bounding box dimensions to: {csv_path}")
    except Exception as e:
        print(f"Failed to save CSV: {e}")
            
    # add wall
    for obj in bpy.data.objects:
        if "dining-room_0/0.wall" in obj.name and all(x not in obj.name for x in exclude_obj):
            
            trimesh_obj=blender_mesh_to_trimesh(obj)
            trimesh_obj_list.append(trimesh_obj)

    cnt=0
    
    polygons_2d=[]
    
    #exclude the wall
    for mesh in trimesh_obj_list[:-1]:
        cnt=cnt+1
        start = time.time()
        try:
        # risky operation
            polygons_2d.append(mesh_to_xy_polygon(mesh))
        except Exception as e:
            #print(f"Skipping item due to error: {e}")
            continue  # skip to next item
        end = time.time()
        #print(f"Elapsed time: {end - start:.4f} seconds",mesh.vertices.shape)
    
    end_preprocessing = time.time()









    captured_obj_idx=[]
    l_cam_x=[x_start]
    l_cam_y=[y_start]
    l_yaw=[math.pi/2]
    l_pitch=[0]
    num_obj=40
    
    max_sight_length=4
    max_samplingcnt=2000
    max_inside=0
    x_resolution=480
    min_dist=0.2
    max_dist=2
    
    cur_cam_x=x_start
    cur_cam_y=y_start
    
    l_cnt_occlusion_check=[]
    l_max_percent_showed=[]
    l_sampling_cnt=[]
    l_cnt_inside_fov=[]
    l_cnt_accessible=[]
    #target_obj_index=[]
    obj_end_time=[]
    obj_start_time=[]

    appearance_obj_list=[]
    
    for obj_cnt in range(len(obj_name_list)):
        #print("object num",obj_cnt )
        max_percent_showed=0
        cnt_occlusion_check=0
        sampling_cnt=0
        cnt_inside_fov=0
        cnt_accessible=0
        
        ##### find closet obj
    
        distance2start=[]
        for i in range( len(polygons_2d)):
            if i not in captured_obj_idx:
                center=polygons_2d[i].centroid
                dis=((cur_cam_x - center.x)**2 + (cur_cam_y - center.y)**2) ** 0.5
                distance2start.append(dis)
            else:
                distance2start.append(100)
                
        cloest = min(distance2start)
        index_cloest = distance2start.index(cloest)
        captured_obj_idx.append(index_cloest)
        
        ##### sample a viewpoint
    
        # scene loading for segmentation map, only need to load once for a target
        scene = pyrender.Scene(bg_color=[0, 0, 0, 0], ambient_light=[1.0, 1.0, 1.0])
        scene_target_obj = pyrender.Scene(bg_color=[0, 0, 0, 0], ambient_light=[1.0, 1.0, 1.0])
        
        for i, mesh in enumerate(trimesh_obj_list):  # mesh_list is your list of trimesh.Trimesh objects
            color = get_unique_color(random.randint(0, (1 << 24) - 1))
            mesh.visual.vertex_colors = np.tile(color, (len(mesh.vertices), 1))
            pm = pyrender.Mesh.from_trimesh(mesh, smooth=False)
            scene.add(pm)
            if i==index_cloest:
                scene_target_obj.add(pm)
                target_color=color
                #print(color)
                
        camera = pyrender.PerspectiveCamera(yfov=ver_fov_rad)
        aspect_ratio = np.tan(hor_fov_rad / 2) / np.tan(ver_fov_rad / 2) # width / height
        y_resolution=math.floor(x_resolution/aspect_ratio)
        
        default_camera_pose = np.array([
            [1.0, 0.0, 0.0, 3],
            [0.0, 1.0, 0.0, 12],
            [0.0, 0.0, 1.0, 10],
            [0.0, 0.0, 0.0, 1.0]
        ])
        camera_node=scene.add(camera, pose=default_camera_pose)
        scene.set_pose(camera_node, pose=default_camera_pose)
        camera_node_target_obj=scene_target_obj.add(camera, pose=default_camera_pose)
        scene.set_pose(camera_node, pose=default_camera_pose)
        
    
        obj_start_time.append(time.time())
        #while (sampling_cnt<max_samplingcnt):
        while (sampling_cnt<500):
            if sampling_cnt%500==0:
                print("###")
            sampling_cnt=sampling_cnt+1
            dis2camera = random.uniform(min_dist, max_dist)
            angle_random= random.uniform(0, math.pi)
            yaw_random=random.uniform(math.pi/4, math.pi/4*3)
            pitch_random=random.uniform(0, math.pi)
    
            obj_cur=polygons_2d[index_cloest]
            new_camera_position= Point(obj_cur.centroid.x+dis2camera * math.cos(angle_random), obj_cur.centroid.y+dis2camera * math.sin(angle_random),height)
            new_camera_position_2d= Point(obj_cur.centroid.x+dis2camera * math.cos(angle_random), obj_cur.centroid.y+dis2camera * math.sin(angle_random))
            ##### if the position in accessible area
            accessibility=True
            if poly_room.buffer(-0.1).contains(new_camera_position_2d)==False:
                accessibility=False
            for o in polygons_2d:
                if o.contains(new_camera_position_2d)==True:
                    accessibility=False
    
            #print("accessibility",accessibility)
            if accessibility==True:
                cnt_accessible=cnt_accessible+1
                #print("accessibile")
                ##### if the target obj in fov
                delta_z=max_sight_length*math.sin(ver_fov_rad/2)
                delta_x=max_sight_length*math.cos(ver_fov_rad/2)*math.sin(hor_fov_rad/2)
                delta_y=max_sight_length*math.cos(ver_fov_rad/2)*math.cos(hor_fov_rad/2)
        
                vertices = np.array([
                    [new_camera_position.x, new_camera_position.y, height],   # cam
                    [new_camera_position.x+delta_x, new_camera_position.y+delta_y, height+delta_z],   # fov 1
                    [new_camera_position.x-delta_x, new_camera_position.y+delta_y, height+delta_z],   # fov 2
                    [new_camera_position.x-delta_x, new_camera_position.y+delta_y, height-delta_z],
                    [new_camera_position.x+delta_x, new_camera_position.y+delta_y, height-delta_z]
                ])
        
                faces = np.array([
                    [0, 1, 2],   # triangle 
                    [0, 2, 3],   # triangle 
                    [0, 3, 4],   # triangle 
                    [0, 4, 1],   # triangle 
                    [1, 2, 3],   # rec
                    [1, 3, 4],   # rec
                ])
        
                #fov without pitch and yaw
                fov_mesh = trimesh.Trimesh(vertices=vertices, faces=faces, process=False)
        
                rotation_pitch = trimesh.transformations.rotation_matrix(
                    pitch_random,               # angle in radians
                    [0, 0, 1],               # axis of rotation (Z-axis)
                    point=[new_camera_position.x, new_camera_position.y, height]     # rotate around the mesh center
                )
        
                rotation_yaw = trimesh.transformations.rotation_matrix(
                    yaw_random-math.pi/2,               # relative to current position
                    [1, 0, 0],               # axis of rotation (X-axis)
                    point=[new_camera_position.x, new_camera_position.y, height]     # rotate around the mesh center
                )
        
                fov_mesh.apply_transform(rotation_yaw).apply_transform(rotation_pitch)
        
                
                # Use mesh_b's vertices or sampled internal points
                points = trimesh_obj_list[index_cloest].vertices  # or: mesh_b.sample(1000)
                
                # Check if all points are inside mesh_a
                inside = fov_mesh.contains(points)
                percent=np.mean(inside) * 100
                if percent>max_inside:
                    max_inside=percent
                    #print(percent)
                # True if all points of mesh_b are inside mesh_a
                is_inside = inside.all()
                
                #print(is_inside)
    
                
                if is_inside:
                    cnt_inside_fov=cnt_inside_fov+1
                    ##### check occlusion using segmentation map
                    #print("###########",sampling_cnt)
                    cnt_occlusion_check=cnt_occlusion_check+1
                    # Create a Rotation object from Euler angles (specify axes order, e.g. 'xyz' or 'zyx')
                    r = R.from_euler('xyz', [yaw_random, 0, pitch_random])
                    # Get the 3x3 rotation matrix
                    R_matrix = r.as_matrix()
                    # Define translation
                    t = np.array([new_camera_position.x, new_camera_position.y, height])
                    # Build the 4x4 transformation matrix
                    T = np.eye(4)
                    T[:3, :3] = R_matrix
                    T[:3, 3] = t
    
    
                    scene.set_pose(camera_node, pose=T)
                    scene_target_obj.set_pose(camera_node_target_obj, pose=T)
    
    
                    r = pyrender.OffscreenRenderer(viewport_width=x_resolution, viewport_height=y_resolution)
                    color, _ = r.render(scene)
                    color_target_obj, _ = r.render(scene_target_obj)
                    plt.savefig(OUTPUT_DIR+"/"+ "visual.pdf")
                    
                    pixels = color_target_obj.reshape(-1, 3)
                    # Get unique RGB rows
                    unique_colors = np.unique(pixels, axis=0)
                    # Optional: convert to list of tuples
                    unique_color_list = [tuple(color) for color in unique_colors]
                    target_color=unique_color_list[-1]
                    
                    mask = np.all(color == target_color, axis=-1)
                    cnt_pixels_rendered=np.sum(mask)
                    mask = np.all(color_target_obj == target_color, axis=-1)
                    cnt_pixels_all=np.sum(mask)
    
                    percent_showed=cnt_pixels_rendered/cnt_pixels_all
                    #print("percent showed",percent_showed)
                    if max_percent_showed<percent_showed:
                        max_percent_showed=percent_showed
                        #print(max_percent_showed)
    
                    if percent_showed>0.7 or cnt_occlusion_check>40:
                        if percent_showed> 0.7:
                            cur_cam_x=new_camera_position.x
                            cur_cam_y=new_camera_position.y
                            l_cam_x.append(new_camera_position.x)
                            l_cam_y.append(new_camera_position.y)
                            l_yaw.append(yaw_random)
                            l_pitch.append(pitch_random)

                            img = Image.fromarray(color)
                            filename = f"{OUTPUT_DIR}/segmentation_obj{obj_cnt}_all.png"
                            img.save(filename)

                            img = Image.fromarray(color)
                            filename = f"{OUTPUT_DIR}/segmentation_obj{obj_cnt}_target.png"
                            img.save(filename)

                            appearance_obj_list.append({
                                "name": obj_name_list[index_cloest],
                                "occlusion": percent_showed
                            })

                        break
    
        
        
    
        
    
    
        ##### update
        l_cnt_occlusion_check.append(cnt_occlusion_check)
        l_max_percent_showed.append(max_percent_showed)
        l_sampling_cnt.append(sampling_cnt)
        l_cnt_inside_fov.append(cnt_inside_fov)
        l_cnt_accessible.append(cnt_accessible)
        obj_end_time.append(time.time())
    
    #print("finished")

    csv_path = OUTPUT_DIR+"/"+"object_appearance.csv"

    try:
        with open(csv_path, 'w', newline='') as f:
            # Define headers
            fieldnames = ["name", "occlusion"]
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            
            writer.writeheader()
            writer.writerows(appearance_obj_list)
            
        print(f"appearance: {csv_path}")
    except Exception as e:
        print(f"Failed to save appearance")



    # ---------------------------------------------------------
    # 1. CONFIGURATION
    # --------------------------------------------------------- # Resolution of the pathfinding grid (meters)
    ROBOT_RADIUS = 0.1  # Buffer around obstacles so camera doesn't clip
    
    Z_HEIGHT = height # From your previous variable (1.5)
    
    # Combine obstacles into one collision object for faster checking
    # We assume 'polygons_2d' contains your furniture footprints
    obstacles_combined = unary_union(polygons_2d)
    obstacles_buffered = obstacles_combined.buffer(ROBOT_RADIUS)
    room_eroded = poly_room.buffer(-ROBOT_RADIUS) # Stay inside room
    
    # Prepare geometry for fast "contains" checks
    prep_obstacles = prep(obstacles_buffered)
    prep_room = prep(room_eroded)
    
    
    
    # ---------------------------------------------------------
    # 2. BUILD NAVIGATION GRAPH (Grid)
    # ---------------------------------------------------------
    #print("Building Navigation Graph...")
    G = nx.Graph()
    
    # Get bounds of the room
    minx, miny, maxx, maxy = poly_room.bounds
    
    # Generate grid nodes
    x_coords = np.arange(minx, maxx, GRID_RES)
    y_coords = np.arange(miny, maxy, GRID_RES)
    
    # Add nodes
    valid_nodes = []
    for x in x_coords:
        for y in y_coords:
            if is_valid_point(x, y,prep_room,prep_obstacles):
                node_id = (round(x, 2), round(y, 2))
                G.add_node(node_id)
                valid_nodes.append(node_id)
    
    # Add edges (8-connectivity: horizontal, vertical, diagonal)
    # We only add an edge if the connection doesn't cross an obstacle
    directions = [
        (GRID_RES, 0, 1.0),       # East
        (-GRID_RES, 0, 1.0),      # West
        (0, GRID_RES, 1.0),       # North
        (0, -GRID_RES, 1.0),      # South
        (GRID_RES, GRID_RES, 1.414),   # NE
        (GRID_RES, -GRID_RES, 1.414),  # SE
        (-GRID_RES, GRID_RES, 1.414),  # NW
        (-GRID_RES, -GRID_RES, 1.414)  # SW
    ]
    
    for node in valid_nodes:
        x, y = node
        for dx, dy, weight in directions:
            neighbor = (round(x + dx, 2), round(y + dy, 2))
            
            if neighbor in G.nodes:
                # Optional: Strict line-of-sight check for diagonal edges
                # to prevent cutting through corners
                # line = LineString([node, neighbor])
                # if not prep_obstacles.intersects(line):
                
                # Simple distance weight
                G.add_edge(node, neighbor, weight=weight)
    
    #print(f"Graph built: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges.")
    
    
    # ---------------------------------------------------------
    # 4. GENERATE TRAJECTORY (TRANSLATE -> ROTATE)
    # ---------------------------------------------------------
    # full_trajectory will contain dictionaries: {'x', 'y', 'z', 'pitch', 'yaw', 'action'}
    full_trajectory = []
    
    # Number of frames to interpolate rotation
    
    
    for i in range(len(l_cam_x) - 1):
        curr_x, curr_y = l_cam_x[i], l_cam_y[i]
        next_x, next_y = l_cam_x[i+1], l_cam_y[i+1]
        
        curr_yaw, curr_pitch = l_yaw[i], l_pitch[i]
        next_yaw, next_pitch = l_yaw[i+1], l_pitch[i+1]
        
        #print(f"Planning segment {i}: ({curr_x:.1f}, {curr_y:.1f}) -> ({next_x:.1f}, {next_y:.1f})")
        
        # A. TRANSLATION PHASE (Move along grid path)
        # -------------------------------------------
        path_points = compute_path_segment(curr_x, curr_y, next_x, next_y,G)
        
        # If path is empty (start==end), skip translation
        if len(path_points) > 0:
            for p in path_points:
                full_trajectory.append({
                    'x': p[0], 'y': p[1], 'z': Z_HEIGHT,
                    'yaw': curr_yaw,   # Maintain previous orientation while moving
                    'pitch': curr_pitch,
                    'action': 'translate'
                })
                
        # B. ROTATION PHASE (Stationary turn at destination)
        # --------------------------------------------------
        # Linearly interpolate angles from curr to next
        # Note: Shortest path interpolation for angles (handle 0/360 wrap) is omitted for brevity 
        # but recommended for production.
        for step in range(1, ROTATION_STEPS + 1):
            alpha = step / ROTATION_STEPS
            interp_yaw = curr_yaw + (next_yaw - curr_yaw) * alpha
            interp_pitch = curr_pitch + (next_pitch - curr_pitch) * alpha
            
            full_trajectory.append({
                'x': next_x, 'y': next_y, 'z': Z_HEIGHT, # Stay at dest
                'yaw': interp_yaw,
                'pitch': interp_pitch,
                'action': 'rotate'
            })
    
    #print(f"Trajectory generated with {len(full_trajectory)} frames.")
    
    # ---------------------------------------------------------
    # 5. VISUALIZATION
    # ---------------------------------------------------------
    plt.figure(figsize=(10, 10))
    
    # Plot Room
    rx, ry = poly_room.exterior.xy
    plt.plot(rx, ry, 'k-', linewidth=2, label='Room')
    
    # Plot Obstacles
    for poly in polygons_2d:
        if not poly.is_empty:
            if poly.geom_type == 'Polygon':
                ox, oy = poly.exterior.xy
                plt.fill(ox, oy, color='gray', alpha=0.5)
            elif poly.geom_type == 'MultiPolygon':
                for sub_p in poly.geoms:
                    ox, oy = sub_p.exterior.xy
                    plt.fill(ox, oy, color='gray', alpha=0.5)
    
    # Plot Trajectory
    traj_x = [t['x'] for t in full_trajectory]
    traj_y = [t['y'] for t in full_trajectory]
    plt.plot(traj_x, traj_y, 'b.-', markersize=2, label='Computed Path')
    
    # Plot Keypoints
    plt.plot(l_cam_x, l_cam_y, 'ro', markersize=8, label='Keypoints')
    
    plt.title("Dijkstra Path Planning (Translate -> Rotate)")
    plt.axis('equal')
    plt.legend()
   
    #filepath = os.path.join(OUTPUT_DIR, "visual.pdf")
    plt.savefig(OUTPUT_DIR+"/"+ "visual.pdf")
    #plt.show()

        # ---------------------------------------------------------
    # 2. APPLY TO ANIMATION
    # ---------------------------------------------------------
    # Get the camera (Ensure we get the one from the outliner)
    cam = bpy.data.objects.get("camera_0_0") # Try specific name from your image
    if not cam:
        cam = bpy.context.scene.camera # Fallback to active


    # Force Euler mode to ensure rotations apply correctly
    if cam.rotation_mode != 'XYZ':
        cam.rotation_mode = 'XYZ'
    
    # Clear any parent inverse correction issues or constraints if necessary
    # (Optional: Only do this if the camera flies wildly off)
    # if cam.parent:
    #    cam.matrix_parent_inverse.identity()
    
    bpy.context.scene.frame_start = 0
    bpy.context.scene.frame_end = len(full_trajectory) - 1

    output_csv =OUTPUT_DIR+"/"+ "trajectory_data.csv"
    
    # Ensure there is data to save
    if len(full_trajectory) > 0:
        # Get column headers from the first dictionary keys
        keys = full_trajectory[0].keys()
    
        with open(output_csv, 'w', newline='') as f:
            dict_writer = csv.DictWriter(f, fieldnames=keys)
            dict_writer.writeheader()
            dict_writer.writerows(full_trajectory)
    
        print(f"Successfully saved {len(full_trajectory)} rows to {os.path.abspath(output_csv)}")
    else:
        print("Trajectory is empty, nothing to save.")
    
    for i, state in enumerate(full_trajectory):
        bpy.context.scene.frame_set(i)
        
        # Unpack trajectory
        x, y, z = state['x'], state['y'], state['z']
        yaw, pitch = state['yaw'], state['pitch']
        """
        # 1. POSITION FIX: Use matrix_world to ignore parent offsets
        target_pos = mathutils.Vector((x, y, z))
        
        # 2. ROTATION FIX: Calculate correct matrix for Blender Camera
        # Note: We reconstruct the matrix entirely to ensure correct orientation
        new_matrix = get_camera_rotation_matrix(target_pos, yaw, pitch)
        
        # Apply to camera
        cam.matrix_world = new_matrix
        """
        # 1. ROTATION: Match PyRender Logic (Euler XYZ)
        # This ensures the render matches your segmentation maps
        rot_euler = mathutils.Euler((yaw, 0, pitch), 'XYZ')
        
        # 2. CONSTRUCT MATRIX WORLD
        mat_rot = rot_euler.to_matrix().to_4x4()
        mat_loc = mathutils.Matrix.Translation((x, y, z))
        
        # Combine and Apply
        cam.matrix_world = mat_loc @ mat_rot
        # Keyframe Matrix (VisualLocRot handles the matrix decomposition automatically)
        cam.keyframe_insert(data_path="location", index=-1)
        cam.keyframe_insert(data_path="rotation_euler", index=-1)

    
    #print("Camera trajectory corrected using World Matrix.")



    # Ensure output directory exists
    output_path_abs = OUTPUT_DIR
    if not os.path.exists(output_path_abs):
        os.makedirs(output_path_abs)
    
    # ---------------------------------------------------------
    # 2. SETUP CYCLES RENDERER
    # ---------------------------------------------------------
    scene = bpy.context.scene
    scene.render.engine = 'CYCLES'
    
    # Set Device (GPU if available, else CPU)
    prefs = bpy.context.preferences.addons['cycles'].preferences
    try:
        # Try to set CUDA or OPTIX if available
        prefs.compute_device_type = 'CUDA' 
        prefs.get_devices()
    except:
        pass
    scene.cycles.device = 'GPU'
    
    # Apply User Settings
    scene.cycles.samples = 64
    scene.cycles.use_adaptive_sampling = True
    scene.cycles.adaptive_threshold = 0.01
    scene.cycles.use_denoising = True
    scene.view_settings.exposure = 1.0
    
    # Resolution (optional, matches your notebook settings if needed)
    scene.render.resolution_x = 1920
    scene.render.resolution_y = 1080
    scene.render.resolution_percentage = 100

    
    scene.frame_step = 3
    
    
    # ---------------------------------------------------------
    # 3. RENDER FRAMES (PNG SEQUENCE)
    # ---------------------------------------------------------
    #print("Starting Frame Render...")
    
    # Set output format to PNG
    scene.render.image_settings.file_format = 'PNG'
    scene.render.filepath = os.path.join(OUTPUT_DIR, FRAME_FILENAME)
    
    # Render the animation
    # This saves frame_0000.png, frame_0001.png, etc.
    bpy.ops.render.render(animation=True)
    
    #print("Frames rendered successfully.")
    
    # ---------------------------------------------------------
    # 4. GENERATE VIDEO (FROM FRAMES)
    # ---------------------------------------------------------
    #print("Compiling Video...")
    
    # Switch to Video Sequence Editor
    """
    if not scene.sequence_editor:
        scene.sequence_editor_create()
        
    seq = scene.sequence_editor.sequences
    
    # Clear existing strips
    for s in list(seq):
        seq.remove(s)
    
    # Add the rendered image sequence to the VSE
    frame_start = scene.frame_start
    frame_end = scene.frame_end
    filenames = [f"{FRAME_FILENAME}{i:04d}.png" for i in range(frame_start, frame_end + 1)]
    
    # Import images as a strip
    # Note: We assume the files were just created in output_path_abs
    files = [{"name": fname} for fname in filenames]
    strip = seq.new_image(
        name="RenderedStrip",
        filepath=os.path.join(output_path_abs, filenames[0]),
        channel=1,
        frame_start=frame_start
    )
    
    # Important: add all files to the strip
    for f in filenames:
        strip.elements.append(f)
    
    # Change Render Settings for Video Output
    scene.render.image_settings.file_format = 'FFMPEG'
    scene.render.ffmpeg.format = 'MPEG4'
    scene.render.ffmpeg.codec = 'H264'
    scene.render.ffmpeg.constant_rate_factor = 'MEDIUM'  # Quality
    scene.render.filepath = os.path.join(OUTPUT_DIR, VIDEO_FILENAME)
    
    # Render the video file (using the sequencer we just set up)
    # We use 'write_still=False' because we are rendering a video file now
    bpy.ops.render.render(animation=True)
    
    # Restore settings (optional)
    scene.render.image_settings.file_format = 'PNG'
    scene.render.filepath = os.path.join(OUTPUT_DIR, FRAME_FILENAME)
    
    #print(f"Video saved to {os.path.join(output_path_abs, VIDEO_FILENAME)}")
    """

In [ ]:
from pathlib import Path

# 1. Define the path using 'r' (raw string) to handle backslashes correctly
folder_path = Path(r"G:\backup\living")

# 2. Use glob to find all files ending in .blend
# This creates a generator of Path objects
blend_files = folder_path.glob("*.blend")

# 3. Print or process them
for file in blend_files:
    file_name=str(file)
    room_type="living-room_0/0.ceiling"
    OUTPUT_DIR = "G:/backup/living"+"/" + file.name  # Relative path (creates folder next to .blend file)
    OUTPUT_DIR=OUTPUT_DIR[:-6]
    FRAME_FILENAME = "frame_"
    VIDEO_FILENAME = "trajectory_video.mp4"
    GRID_RES = 0.1
    ROTATION_STEPS = 3
    height=1.5
        
    print("##################################################\n")
    print(file_name)
    folder_path = Path(OUTPUT_DIR)
    folder_path.mkdir(parents=True, exist_ok=True)

    camera_traj(file_name,
                room_type,
                OUTPUT_DIR,
                FRAME_FILENAME,
                VIDEO_FILENAME,
                GRID_RES,
                ROTATION_STEPS,
                height
               )

##################################################

G:\backup\living\scene (1).blend
Successfully saved bounding box dimensions to: G:/backup/living/scene (1)/object_bbox_dimensions.csv
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
appearance: G:/backup/living/scene (1)/object_appearance.csv
No path found between (8.870227770976326,14.143349726026141) and (4.3158613981551355,8.23600809031156)
Successfully saved 276 rows to G:\backup\living\scene (1)\trajectory_data.csv
##################################################

G:\backup\living\scene (10).blend
Successfully saved bounding box dimensions to: G:/backup/living/scene (10)/object_bbox_dimensions.csv
###
###
###
###
###
###
###
###
###
###
###
###


In [ ]:
"G:/backup/living"+"/" + file.name

In [ ]:
folder_path = Path(OUTPUT_DIR[:-6])
folder_path.mkdir(parents=True, exist_ok=True)